# 机器学习基础 - 01

本笔记本介绍机器学习的基本概念和算法，使用旅游数据作为实例。

## 学习目标
- 了解监督学习、无监督学习的概念
- 掌握线性回归、分类、聚类基础算法
- 学会使用scikit-learn进行机器学习建模
- 应用机器学习解决旅游相关问题

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境设置完成！")

## 1. 生成示例旅游数据

我们首先创建一些模拟的旅游数据来进行学习。

In [ ]:
# 设置随机种子以确保结果可重复
np.random.seed(42)

# 生成旅游景点数据
n_attractions = 200

data = {
    'attraction_id': range(n_attractions),
    'category': np.random.choice(['历史', '自然', '现代', '文化', '美食'], n_attractions),
    'rating': np.random.normal(4.0, 0.8, n_attractions).clip(1, 5),
    'price_level': np.random.choice([1, 2, 3, 4], n_attractions, p=[0.3, 0.4, 0.2, 0.1]),
    'visit_duration': np.random.exponential(3, n_attractions) + 1,  # 小时
    'distance_from_center': np.random.exponential(10, n_attractions),  # 公里
    'visitor_count': np.random.poisson(1000, n_attractions) + 100,
    'season_score': np.random.uniform(0.5, 1.0, n_attractions)
}

# 创建DataFrame
df = pd.DataFrame(data)

# 根据一些逻辑关系调整数据，使其更realistic
df['rating'] = df['rating'] + (df['price_level'] - 2) * 0.1 + np.random.normal(0, 0.1, n_attractions)
df['rating'] = df['rating'].clip(1, 5)

print("数据生成完成！")
print(f"数据集大小: {df.shape}")
df.head()

## 2. 数据探索性分析

让我们先了解一下数据的分布和特征。

In [ ]:
# 查看数据基本信息
print("数据集信息:")
print(df.info())
print("\n数据统计摘要:")
print(df.describe())

In [ ]:
# 可视化数据分布
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('旅游景点数据分析', fontsize=16)

# 评分分布
axes[0, 0].hist(df['rating'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('评分分布')
axes[0, 0].set_xlabel('评分')
axes[0, 0].set_ylabel('频次')

# 类别分布
category_counts = df['category'].value_counts()
axes[0, 1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
axes[0, 1].set_title('景点类别分布')

# 价格等级分布
price_counts = df['price_level'].value_counts().sort_index()
axes[0, 2].bar(price_counts.index, price_counts.values)
axes[0, 2].set_title('价格等级分布')
axes[0, 2].set_xlabel('价格等级')
axes[0, 2].set_ylabel('景点数量')

# 游览时长分布
axes[1, 0].hist(df['visit_duration'], bins=20, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('游览时长分布')
axes[1, 0].set_xlabel('时长(小时)')
axes[1, 0].set_ylabel('频次')

# 距离中心距离分布
axes[1, 1].hist(df['distance_from_center'], bins=20, edgecolor='black', alpha=0.7)
axes[1, 1].set_title('距离市中心距离分布')
axes[1, 1].set_xlabel('距离(公里)')
axes[1, 1].set_ylabel('频次')

# 评分vs价格等级关系
sns.boxplot(data=df, x='price_level', y='rating', ax=axes[1, 2])
axes[1, 2].set_title('不同价格等级的评分分布')
axes[1, 2].set_xlabel('价格等级')
axes[1, 2].set_ylabel('评分')

plt.tight_layout()
plt.show()

## 3. 监督学习 - 回归问题

我们使用线性回归来预测景点的评分。

In [ ]:
# 准备特征和目标变量
# 对类别特征进行编码
df_encoded = pd.get_dummies(df, columns=['category'], prefix='category')

# 选择特征
feature_cols = ['price_level', 'visit_duration', 'distance_from_center', 
                'visitor_count', 'season_score'] + [col for col in df_encoded.columns if col.startswith('category_')]

X = df_encoded[feature_cols]
y = df_encoded['rating']

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")
print(f"特征列表: {feature_cols}")

In [ ]:
# 训练线性回归模型
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 预测
y_pred = lr_model.predict(X_test)

# 评估模型
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"均方误差 (MSE): {mse:.4f}")
print(f"均方根误差 (RMSE): {rmse:.4f}")
print(f"模型R²得分: {lr_model.score(X_test, y_test):.4f}")

In [ ]:
# 可视化预测结果
plt.figure(figsize=(12, 5))

# 实际值vs预测值散点图
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('实际评分')
plt.ylabel('预测评分')
plt.title('实际值vs预测值')

# 特征重要性
plt.subplot(1, 2, 2)
feature_importance = pd.Series(lr_model.coef_, index=feature_cols).sort_values()
feature_importance.plot(kind='barh')
plt.title('特征重要性（回归系数）')
plt.xlabel('回归系数')

plt.tight_layout()
plt.show()

## 4. 监督学习 - 分类问题

我们将评分分为高（>4分）和低（≤4分）两类，进行分类预测。

In [ ]:
# 创建分类标签
y_class = (df_encoded['rating'] > 4).astype(int)  # 1表示高评分，0表示低评分

# 分割数据
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X, y_class, test_size=0.2, random_state=42)

# 训练逻辑回归分类器
clf_model = LogisticRegression(random_state=42, max_iter=1000)
clf_model.fit(X_train_cls, y_train_cls)

# 预测
y_pred_cls = clf_model.predict(X_test_cls)
y_pred_proba = clf_model.predict_proba(X_test_cls)[:, 1]  # 预测为高评分的概率

# 评估
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"分类准确率: {accuracy:.4f}")

# 分类报告
from sklearn.metrics import classification_report, confusion_matrix
print("\n分类报告:")
print(classification_report(y_test_cls, y_pred_cls, target_names=['低评分', '高评分']))

# 混淆矩阵
cm = confusion_matrix(y_test_cls, y_pred_cls)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['低评分', '高评分'], yticklabels=['低评分', '高评分'])
plt.title('混淆矩阵')
plt.ylabel('实际标签')
plt.xlabel('预测标签')
plt.show()

## 5. 无监督学习 - 聚类分析

使用K-Means聚类来发现景点的不同类型。

In [ ]:
# 选择用于聚类的特征（数值型特征）
cluster_features = ['rating', 'price_level', 'visit_duration', 'distance_from_center', 'visitor_count']
X_cluster = df[cluster_features]

# 标准化特征
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# 使用肘部法则选择最佳聚类数
inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# 绘制肘部图
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-')
plt.title('肘部法则选择最佳聚类数')
plt.xlabel('聚类数 (k)')
plt.ylabel('簇内平方和 (Inertia)')
plt.grid(True)
plt.show()

In [ ]:
# 使用K=4进行聚类
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# 将聚类结果添加到原数据
df['cluster'] = cluster_labels

# 分析各聚类的特征
print("各聚类的特征分析:")
cluster_summary = df.groupby('cluster')[cluster_features + ['category']].agg({
    'rating': 'mean',
    'price_level': 'mean', 
    'visit_duration': 'mean',
    'distance_from_center': 'mean',
    'visitor_count': 'mean',
    'category': lambda x: x.mode().iloc[0]  # 最频繁的类别
}).round(2)

print(cluster_summary)

In [ ]:
# 可视化聚类结果
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('聚类结果可视化', fontsize=16)

# 评分vs价格等级
scatter1 = axes[0, 0].scatter(df['price_level'], df['rating'], c=df['cluster'], cmap='viridis', alpha=0.6)
axes[0, 0].set_xlabel('价格等级')
axes[0, 0].set_ylabel('评分')
axes[0, 0].set_title('评分 vs 价格等级')
plt.colorbar(scatter1, ax=axes[0, 0])

# 游览时长vs距离
scatter2 = axes[0, 1].scatter(df['distance_from_center'], df['visit_duration'], c=df['cluster'], cmap='viridis', alpha=0.6)
axes[0, 1].set_xlabel('距离市中心(公里)')
axes[0, 1].set_ylabel('游览时长(小时)')
axes[0, 1].set_title('游览时长 vs 距离市中心')
plt.colorbar(scatter2, ax=axes[0, 1])

# 聚类大小
cluster_counts = df['cluster'].value_counts().sort_index()
axes[1, 0].bar(cluster_counts.index, cluster_counts.values, color=['red', 'green', 'blue', 'orange'])
axes[1, 0].set_xlabel('聚类编号')
axes[1, 0].set_ylabel('景点数量')
axes[1, 0].set_title('各聚类景点数量')

# 聚类vs原始类别的关系
crosstab = pd.crosstab(df['cluster'], df['category'])
crosstab_pct = crosstab.div(crosstab.sum(axis=1), axis=0) * 100
sns.heatmap(crosstab_pct, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1, 1])
axes[1, 1].set_title('聚类 vs 原始类别 (百分比)')
axes[1, 1].set_xlabel('原始类别')
axes[1, 1].set_ylabel('聚类编号')

plt.tight_layout()
plt.show()

## 6. 实际应用：智能推荐系统

结合我们学到的机器学习技术，构建一个简单的景点推荐系统。

In [ ]:
class TravelRecommendationSystem:
    def __init__(self, data, rating_model, cluster_model, scaler):
        self.data = data
        self.rating_model = rating_model
        self.cluster_model = cluster_model
        self.scaler = scaler
        
    def recommend_by_preference(self, preferred_category=None, max_distance=None, 
                               min_rating=None, max_price_level=None, top_k=5):
        """
        基于用户偏好推荐景点
        """
        filtered_data = self.data.copy()
        
        # 应用筛选条件
        if preferred_category:
            filtered_data = filtered_data[filtered_data['category'] == preferred_category]
        if max_distance:
            filtered_data = filtered_data[filtered_data['distance_from_center'] <= max_distance]
        if min_rating:
            filtered_data = filtered_data[filtered_data['rating'] >= min_rating]
        if max_price_level:
            filtered_data = filtered_data[filtered_data['price_level'] <= max_price_level]
            
        if len(filtered_data) == 0:
            return "没有符合条件的景点"
            
        # 按评分排序并返回前k个
        recommendations = filtered_data.nlargest(top_k, 'rating')
        
        return recommendations[['attraction_id', 'category', 'rating', 'price_level', 
                              'distance_from_center', 'visit_duration', 'cluster']]
    
    def recommend_similar_attractions(self, attraction_id, top_k=5):
        """
        基于聚类推荐相似景点
        """
        if attraction_id not in self.data['attraction_id'].values:
            return "景点ID不存在"
            
        # 获取目标景点的聚类
        target_cluster = self.data[self.data['attraction_id'] == attraction_id]['cluster'].iloc[0]
        
        # 找到同一聚类的其他景点
        similar_attractions = self.data[
            (self.data['cluster'] == target_cluster) & 
            (self.data['attraction_id'] != attraction_id)
        ]
        
        if len(similar_attractions) == 0:
            return "没有相似的景点"
            
        # 按评分排序
        recommendations = similar_attractions.nlargest(top_k, 'rating')
        
        return recommendations[['attraction_id', 'category', 'rating', 'price_level', 
                              'distance_from_center', 'visit_duration']]

# 创建推荐系统实例
recommender = TravelRecommendationSystem(df, lr_model, kmeans, scaler)

print("推荐系统已创建！")

In [ ]:
# 测试推荐系统
print("=== 基于偏好的推荐 ===")
print("\n推荐高评分的历史类景点（距离市中心10公里内）:")
recommendations1 = recommender.recommend_by_preference(
    preferred_category='历史', max_distance=10, min_rating=4.0, top_k=3
)
print(recommendations1)

print("\n=== 基于相似性的推荐 ===")
print("与景点ID为5的景点相似的推荐:")
recommendations2 = recommender.recommend_similar_attractions(attraction_id=5, top_k=3)
print(recommendations2)

# 显示目标景点信息
target_info = df[df['attraction_id'] == 5][['attraction_id', 'category', 'rating', 'price_level', 'cluster']]
print("\n目标景点信息:")
print(target_info)

## 总结

在这个笔记本中，我们学习了:

1. **数据探索**: 理解旅游数据的分布和特征
2. **监督学习**:
   - 回归：预测景点评分
   - 分类：预测高/低评分景点
3. **无监督学习**:
   - 聚类：发现景点的隐含分组
4. **实际应用**: 构建智能推荐系统

### 下一步学习建议:
- 尝试更复杂的算法（随机森林、梯度提升等）
- 学习特征工程技术
- 探索深度学习在推荐系统中的应用
- 学习模型评估和调参技术

### 练习任务:
1. 尝试使用不同的机器学习算法，比较它们的性能
2. 添加更多特征，观察模型性能的变化
3. 实现基于协同过滤的推荐算法
4. 为推荐系统添加用户评价反馈机制